In [1]:
import scanpy as sc
import pandas as pd
from sklearn import metrics
import torch

import matplotlib.pyplot as plt
import seaborn as sns

import os
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import SEDR

In [22]:
random_seed = 2023
SEDR.fix_seed(random_seed)

# gpu
device = 'cuda:0' if torch.cuda.is_available() else 'cpu'

## Load datasets

In [2]:
Xenium_count = pd.read_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/data/Xenium/tile_23/tile_23_count.csv', index_col=0)
Xenium_meta = pd.read_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/data/Xenium/tile_23/tile_23_meta_update.csv', index_col=0)

In [3]:
cell_count = Xenium_meta['Cluster'].value_counts()
Xenium_meta['state_rank'] = 0
for i, cell_type in enumerate(cell_count.index):
    Xenium_meta.loc[Xenium_meta['Cluster'] == cell_type, 'state_rank'] = i + 1

Xenium_meta['x_centroid'] = 100 * (Xenium_meta['x_centroid'] - Xenium_meta['x_centroid'].min()) / (Xenium_meta['x_centroid'].max() - Xenium_meta['x_centroid'].min())
Xenium_meta['y_centroid'] = 100 * (Xenium_meta['y_centroid'] - Xenium_meta['y_centroid'].min()) / (Xenium_meta['y_centroid'].max() - Xenium_meta['y_centroid'].min())

In [4]:
simspace_count = pd.read_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/output/scdesign/tile_23/tile_23_count_test.csv', index_col=0)
simspace_meta = pd.read_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/output/scdesign/tile_23/tile_23_meta_update_test.csv', index_col=0)
simspace_count = simspace_count.T

In [5]:
simspace_meta['row'] = 100 * (simspace_meta['row'] - simspace_meta['row'].min()) / (simspace_meta['row'].max() - simspace_meta['row'].min())
simspace_meta['col'] = 100 * (simspace_meta['col'] - simspace_meta['col'].min()) / (simspace_meta['col'].max() - simspace_meta['col'].min())
simspace_meta['row'] = simspace_meta['row'] + 200

In [6]:
sccube_count = pd.read_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/reproduce/simulation/scCube/data/Xenium_Breast_23_sc_data.csv', index_col=0)
sccube_meta = pd.read_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/reproduce/simulation/scCube/data/Xenium_Breast_23_sc_meta.csv', index_col=0)

In [7]:
sccube_meta['point_x'] = 100 * (sccube_meta['point_x'] - sccube_meta['point_x'].min()) / (sccube_meta['point_x'].max() - sccube_meta['point_x'].min())
sccube_meta['point_y'] = 100 * (sccube_meta['point_y'] - sccube_meta['point_y'].min()) / (sccube_meta['point_y'].max() - sccube_meta['point_y'].min())
sccube_meta['point_x'] = sccube_meta['point_x'] + 100

In [8]:
concat_count = pd.concat([Xenium_count, simspace_count, sccube_count], axis=1)
Xenium_meta = Xenium_meta[['x_centroid', 'y_centroid', 'Cluster']]
simspace_meta = simspace_meta[['row', 'col', 'fitted_celltype']]
sccube_meta = sccube_meta[['point_x', 'point_y', 'Cell_type']]
Xenium_meta.columns = ['x_centroid', 'y_centroid', 'Cluster']
simspace_meta.columns = ['x_centroid', 'y_centroid', 'Cluster']
sccube_meta.columns = ['x_centroid', 'y_centroid', 'Cluster']
Xenium_meta['dataset'] = 'Xenium'
simspace_meta['dataset'] = 'SimSpace'
sccube_meta['dataset'] = 'scCube'
concat_meta = pd.concat([Xenium_meta, simspace_meta, sccube_meta], axis=0)
concat_meta.index = concat_count.columns

In [16]:
adata = sc.AnnData(X=concat_count.T, obs=concat_meta)

## SEDR

In [17]:
adata.layers['count'] = adata.X.copy()

In [18]:
sc.pp.normalize_total(adata, target_sum=1e6)
sc.pp.scale(adata)

In [20]:
adata.obsm['spatial'] = adata.obs[['x_centroid', 'y_centroid']].values

In [21]:
graph_dict = SEDR.graph_construction(adata, 12)
print(graph_dict)

{'adj_norm': tensor(indices=tensor([[   0,    0,    0,  ..., 6936, 6936, 6936],
                       [   0,    1,    2,  ..., 6785, 6787, 6936]]),
       values=tensor([0.0769, 0.0769, 0.0741,  ..., 0.0606, 0.0606, 0.0625]),
       size=(6937, 6937), nnz=100659, layout=torch.sparse_coo), 'adj_label': tensor(indices=tensor([[   0,    0,    0,  ..., 6936, 6936, 6936],
                       [   0,    1,    2,  ..., 6785, 6787, 6936]]),
       values=tensor([1., 1., 1.,  ..., 1., 1., 1.]),
       size=(6937, 6937), nnz=100659, dtype=torch.float64,
       layout=torch.sparse_coo), 'norm_value': 0.5010480659523866}


In [24]:
from sklearn.decomposition import PCA  # sklearn PCA is used because PCA in scanpy is not stable.
adata_X = PCA(n_components=200, random_state=42).fit_transform(adata.X)
adata.obsm['X_pca'] = adata_X

In [25]:
sedr_net = SEDR.Sedr(adata.obsm['X_pca'], graph_dict, mode='clustering', device=device)
using_dec = True
if using_dec:
    sedr_net.train_with_dec(N=1)
else:
    sedr_net.train_without_dec(N=1)
sedr_feat, _, _, _ = sedr_net.process()
adata.obsm['SEDR'] = sedr_feat

100%|██████████| 200/200 [00:16<00:00, 11.99it/s]


In [33]:
import umap

reducer = umap.UMAP(random_state=random_seed)
umap_embedding = reducer.fit_transform(adata.obsm['SEDR'])
adata.obsm['SEDR_umap'] = umap_embedding

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [36]:
# Save UMAP embedding to CSV
umap_df = pd.DataFrame(umap_embedding, index=concat_meta.index, columns=['UMAP1', 'UMAP2'])
umap_df['Dataset'] = concat_meta['dataset']
umap_df['Cluster'] = concat_meta['Cluster']
umap_df.to_csv('/Users/zhaotianxiao/Library/CloudStorage/Dropbox/FenyoLab/Project/Spatialsim/reproduce/clustering/SEDR/umap_embedding.csv')